In [1]:
%load_ext autoreload
%autoreload 2

import sys
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../../..")

import lib.ehr.example_datasets.mimiciv_aki as m4aki

from lib.utils import modified_environ, write_config

In [2]:
# create a database connection
sqluser = 'postgres'
dbname = 'mimiciv'
hostname = 'localhost'
password = 'qwerasdf'
port = '5432'

url = f'postgresql+psycopg2://{sqluser}:{password}@{hostname}:{port}/{dbname}'

with modified_environ(MIMICIV_URL=url):
    mimiciv_config = m4aki.AKIMIMICIVDatasetConfig()
    tvx_config = m4aki.TVxAKIMIMICIVDatasetConfig()

    write_config(mimiciv_config.to_dict(), 'dataset_mimiciv_aki_config.json')
    write_config(tvx_config.to_dict(), 'tvx_mimiciv_aki_config.json')


In [3]:
with modified_environ(MIMICIV_URL=url):
    dataset = m4aki.AKIMIMICIVDataset(config=mimiciv_config)           

	33  10:5A1D60Z  Performance of Urinary Filtration, Multiple          33          10  5A1D60Z
	33  10:5A1D60Z                              Performance of Urinary Filtration, Multiple          33          10  5A1D60Z
	38     9:00.40                                               Procedure on single vessel          38           9    00.40
	39     9:00.41                                                 Procedure on two vessels          39           9    00.41
	40     9:00.45                                          Insertion of one vascular stent          40           9    00.45
	41     9:00.46                                         Insertion of two vascular stents          41           9    00.46
	50     9:36.06                   Insertion of non-drug-eluting coronary artery stent(s)          50           9    36.06
	51     9:36.07                       Insertion of drug-eluting coronary artery stent(s)          51           9    36.07
	70     9:39.90  Insertion of non-drug-eluting perip

In [4]:
dataset = dataset.execute_pipeline()

Transforming Dataset:   0%|          | 0/10 [00:00<?, ?transformations/s]

In [5]:
tvx = m4aki.TVxAKIMIMICIVDataset(config=tvx_config, dataset=dataset)

In [19]:
from lib.ehr.tvx_ehr import TVxEHRScheme

target_scheme = TVxEHRScheme(config=tvx.config.scheme, context_view=tvx.dataset.scheme_manager.view())

In [23]:
target_scheme.config.scheme_fields()

{'gender': 'mimiciv.aki_study.gender',
 'ethnicity': 'mimiciv.aki_study.target_ethnicity',
 'dx_discharge': 'dx_icd9',
 'obs': 'mimiciv.aki_study.obs',
 'icu_inputs': 'mimiciv.aki_study.target_icu_inputs',
 'icu_procedures': 'mimiciv.aki_study.target_icu_procedures',
 'hosp_procedures': 'mimiciv.aki_study.target_hosp_procedures'}

In [25]:
target_scheme.context_view.scheme.keys()#['mimiciv.aki_study.target_ethnicity']

dict_keys(['dx_icd9', 'dx_icd10', 'pr_icd9', 'dx_flat_icd10', 'pr_flat_icd10', 'dx_ccs', 'pr_ccs', 'dx_flat_ccs', 'pr_flat_ccs', 'mimiciv.aki_study.gender', 'mimiciv.aki_study.ethnicity', 'mimiciv.aki_study.target_mimiciv.aki_study.ethnicity', 'mimiciv.aki_study.icu_inputs', 'mimiciv.aki_study.target_mimiciv.aki_study.icu_inputs', 'mimiciv.aki_study.icu_procedures', 'mimiciv.aki_study.target_mimiciv.aki_study.icu_procedures', 'mimiciv.aki_study.hosp_procedures', 'mimiciv.aki_study.target_mimiciv.aki_study.hosp_procedures', 'mimiciv.aki_study.dx_discharge', 'mimiciv.aki_study.obs'])

In [11]:
tvx = tvx.execute_pipeline()

Transforming Dataset:   0%|          | 0/10 [00:00<?, ?transformations/s]

KeyError: 'ethnicity'